# Домашнее задание к вебинару 5. Part-of-Speech Tagger, NER

**Задание 1.** Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы сделать выводы

In [1]:
# pip install pyconll

In [4]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

In [5]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-06-13 07:24:10--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M  --.-KB/s    in 0.1s    

2022-06-13 07:24:11 (321 MB/s) - ‘ru_syntagrus-ud-train-a.conllu’ saved [40736565/40736565]

--2022-06-13 07:24:11--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [6]:
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [7]:
#просматриваем 2 предложения тренировочных данных:
#в каждом из них для каждого токена выводим form (словоформа для чтения) и соответствующую upos (часть речи)
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [8]:
# собираем тренировочную выборку:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
# собираем тестовую выборку:
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

# собираем выборку, где будут только токены   
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [9]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [10]:
fdata_train[:3]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

**UnigramTagger** (учитывает условную частоту тегов и предсказывает наиболее частый тег для каждого токена, не ориентируется на соседние слова)

In [11]:
unigram_tagger = UnigramTagger(fdata_train)
accuracy_unigram = unigram_tagger.evaluate(fdata_test)
display(accuracy_unigram)

0.823732013802982

**BigramTagger** (учитывает тэги двух слов: текущее и предыдущее слово)

In [12]:
bigram_tagger = BigramTagger(fdata_train)
accuracy_bigram = bigram_tagger.evaluate(fdata_test)
display(accuracy_bigram)

0.6093886320724006

**TrigramTagger**

In [13]:
trigram_tagger = TrigramTagger(fdata_train)
accuracy_trigram = trigram_tagger.evaluate(fdata_test)
display(accuracy_trigram)

0.1778631421316492

**Комбинация теггеров**

In [14]:
tagger_combinations = [[UnigramTagger, BigramTagger], 
                       [UnigramTagger, TrigramTagger], 
                       [BigramTagger, TrigramTagger], 
                       [UnigramTagger, BigramTagger, TrigramTagger]]

def backoff_tagger(train_sents, taggers_list, backoff=None):
    for cls in taggers_list:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

accuracy_combs = []
backoff = DefaultTagger('NN') 
for comb in tagger_combinations:
  tag = backoff_tagger(fdata_train,
                       comb,
                       backoff = backoff) 
  accuracy_combs.append(tag.evaluate(fdata_test))

In [15]:
accuracy_combs

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [16]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 
                         'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 
                         'UnigramTagger+BigramTagger+TrigramTagger'], 
              'Accuracy' : [accuracy_unigram, accuracy_bigram, accuracy_trigram] + accuracy_combs}).sort_values(by = 'Accuracy', ascending=False)


In [17]:
result

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


Вывод: наилучший результат достигнут при применении комбинации тэггеров UnigramTagger+BigramTagger+TrigramTagger

In [18]:
#собираем токены и лэйблы в списки

train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])


In [19]:
#кодируем целевые метки значениями от 0 до (кол-во классов - 1)
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [20]:
train_enc_labels

array([ 7, 13,  7, ...,  1, 11, 13])

In [21]:
#строим модели логистической регрессии с различными векторайзерами

vectorizers_char = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 

vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 

n_features = [500, 1000, 2000]

vectorizers_hash_char = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers_char + vectorizers_word + vectorizers_hash_char + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:110000])
    
    lr = LogisticRegression(random_state=42, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:110000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:110000], pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:110000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     10688
         ADP       0.98      1.00      0.99      9819
         ADV       0.92      0.90      0.91      5695
         AUX       0.81      0.97      0.88      1007
       CCONJ       0.88      0.98      0.93      4074
         DET       0.88      0.75      0.81      2841
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     26115
      NO_TAG       1.00      1.00      1.00       195
         NUM       0.86      0.90      0.88      1378
        PART       0.95      0.78      0.86      3600
        PRON       0.83      0.89      0.86      5113
       PROPN       0.78      0.58      0.66      4107
       PUNCT       1.00      1.00      1.00     21011
       SCONJ       0.80      0.91      0.85      2081
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [22]:
result_vectorizers = pd.DataFrame({'Vectorizer': vectorizers_char + vectorizers_word + vectorizers_hash_char + vectorizers_hash_word,
                            'F1_score': f1_scores,
                            'Accuracy': accuracy_scores})
display(result_vectorizers)

,Vectorizer,F1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.926843,0.928582
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.920009,0.922527
2,"HashingVectorizer(analyzer='char', n_features=...",0.881258,0.884200
3,"CountVectorizer(ngram_range=(1, 3))",0.646719,0.638336
4,"TfidfVectorizer(ngram_range=(1, 3))",0.663256,0.653709
5,"HashingVectorizer(n_features=1000, ngram_range...",0.343273,0.363418
6,"HashingVectorizer(analyzer='char', n_features=...",0.844096,0.849564
7,"HashingVectorizer(analyzer='char', n_features=...",0.881258,0.884200
8,"HashingVectorizer(analyzer='char', n_features=...",0.894470,0.898155
9,"HashingVectorizer(n_features=500, ngram_range=...",0.303205,0.336064


In [23]:
result_vectorizers.sort_values('F1_score', ascending=False).head(1)

,Vectorizer,F1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.926843,0.928582


In [24]:
result_vectorizers.sort_values('Accuracy', ascending=False).head(1)

,Vectorizer,F1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.926843,0.928582


In [25]:
result_vectorizers.sort_values('F1_score', ascending=False).tail(1)

,Vectorizer,F1_score,Accuracy
9,"HashingVectorizer(n_features=500, ngram_range=...",0.303205,0.336064


**Вывод**: лучший метод pos tagging (по результатам подсчета метрик) --  теггер с CountVectorizer(ngram_range=(1, 3), analyzer='char')

**Задание 2.** Проверить насколько хорошо работает NER. 
данные брать из http://www.labinform.ru/pub/named_entities/   
проверить NER из nltk и spacy   
написать свой нер попробовать разные подходы  
передаём в сетку токен и его соседей  
передаём в сетку только токен 
свой вариант 
сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [30]:
import os
import corus
from glob import glob
import random
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
from razdel import tokenize 

import matplotlib
%matplotlib inline

import spacy
from spacy import displacy

import csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [31]:
!wget http://www.labinform.ru/pub/named_entities/collection3.zip

--2022-06-13 07:39:52--  http://www.labinform.ru/pub/named_entities/collection3.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1887130 (1.8M) [application/zip]
Saving to: ‘collection3.zip’

collection3.zip     100%[===================>]   1.80M  2.28MB/s    in 0.8s    

2022-06-13 07:39:53 (2.28 MB/s) - ‘collection3.zip’ saved [1887130/1887130]



In [32]:
!unzip collection3.zip

Archive:  collection3.zip
   creating: Collection3/
  inflating: Collection3/001.ann     
  inflating: Collection3/001.txt     
  inflating: Collection3/002.ann     
  inflating: Collection3/002.txt     
  inflating: Collection3/003.ann     
  inflating: Collection3/003.txt     
  inflating: Collection3/004.ann     
  inflating: Collection3/004.txt     
  inflating: Collection3/005.ann     
  inflating: Collection3/005.txt     
  inflating: Collection3/006.ann     
  inflating: Collection3/006.txt     
  inflating: Collection3/007.ann     
  inflating: Collection3/007.txt     
  inflating: Collection3/008.ann     
  inflating: Collection3/008.txt     
  inflating: Collection3/009.ann     
  inflating: Collection3/009.txt     
  inflating: Collection3/010.ann     
  inflating: Collection3/010.txt     
  inflating: Collection3/011.ann     
  inflating: Collection3/011.txt     
  inflating: Collection3/012.ann     
  inflating: Collection3/012.txt     
  inflating: Collection3/013.ann    

In [33]:
# Соберем только текстовые файлы
text_files = sorted([file for file in os.listdir('/content/Collection3/') if file.endswith(".txt")])
text_files[:3]

['001.txt', '002.txt', '003.txt']

In [34]:
#заносим данные файлов txt в датасет
texts = []
for text_file in text_files:
    text = open('/content/Collection3/' + text_file, encoding='utf-8')
    text = text.read()
    texts.append(text)
    
data_text = pd.DataFrame({'text': texts })
data_text

,text
0,Россия рассчитывает на конструктивное воздейст...
1,Комиссар СЕ критикует ограничительную политику...
2,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,4 октября назначены очередные выборы Верховног...
4,Следственное управление при прокуратуре требуе...
...,...
995,"Депутат от ""ЕР"": К отставке А.Сердюкова причас..."
996,\nСи Цзиньпин избран генсеком Коммунистической...
997,"""Ведомости"" узнали о смене лидера московских е..."
998,СМИ узнали о кутежах туркменского чиновника на...


**NLTK NER**

In [35]:
# пример текста 
article = data_text['text'][8]

nltk.pos_tag(nltk.word_tokenize(article))

[('``', '``'),
 ('Норильский', 'NN'),
 ('никель', 'NN'),
 ("''", "''"),
 ('нашел', 'CC'),
 ('советчика', 'NNP'),
 ('Им', 'NNP'),
 ('стал', 'NNP'),
 ('Эдриан', 'NNP'),
 ('Коутс', 'NNP'),
 (',', ','),
 ('помогавший', 'NNP'),
 ('слиянию', 'NNP'),
 ('Mittal', 'NNP'),
 ('и', 'NNP'),
 ('Arcelor', 'NNP'),
 ('У', 'NNP'),
 ('Владимира', 'NNP'),
 ('Стржалковского', 'NNP'),
 ('появится', 'NNP'),
 ('помощник', 'NNP'),
 ('-', ':'),
 ('специалист', 'NN'),
 ('по', 'JJ'),
 ('проведению', 'NNP'),
 ('сделок', 'NNP'),
 ('M', 'NNP'),
 ('&', 'CC'),
 ('A', 'NNP'),
 ("''", "''"),
 ('Норильский', 'NN'),
 ('никель', 'NN'),
 ("''", "''"),
 ('нашел', 'CC'),
 ('частичную', 'NNP'),
 ('замену', 'NNP'),
 ('заместителю', 'NNP'),
 ('гендиректора', 'NNP'),
 ('Таву', 'NNP'),
 ('Моргану', 'NNP'),
 ('.', '.'),
 ('Пока', 'VB'),
 ('его', 'JJ'),
 ('функции', 'NNP'),
 ('в', 'NNP'),
 ('должности', 'NNP'),
 ('главного', 'NNP'),
 ('советника', 'NNP'),
 ('по', 'NNP'),
 ('корпоративной', 'NNP'),
 ('политике', 'NNP'),
 ('будет', 'N

In [36]:
 #распознаем именнованные сущности с помощью классификатора (Person, Organization, GPE)

{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(article))) if hasattr(chunk, 'label')}

{('Arcelor У Владимира Стржалковского', 'PERSON'),
 ('BHP Billiton', 'ORGANIZATION'),
 ('Billiton', 'PERSON'),
 ('HSBC', 'ORGANIZATION'),
 ('IPO Vedanta Resources', 'ORGANIZATION'),
 ('Mittal', 'ORGANIZATION'),
 ('Steel', 'PERSON'),
 ('Баранов', 'PERSON'),
 ('Денисом Морозовым', 'PERSON'),
 ('Коутс', 'PERSON'),
 ('Кроме', 'PERSON'),
 ('Михаиле Прохорове', 'PERSON'),
 ('Эдриан Коутс', 'PERSON'),
 ('Эдриана Коутса', 'PERSON')}

In [37]:
pd.read_csv('/content/Collection3/009.ann', delimiter='\t')

,T1,ORG 1 18,Норильский никель
0,T2,PER 47 59,Эдриан Коутс
1,T3,ORG 80 86,Mittal
2,T4,ORG 89 96,Arcelor
3,T5,PER 103 127,Владимира Стржалковского
4,T6,ORG 188 205,Норильский никель
5,T7,PER 255 267,Таву Моргану
6,T8,PER 359 371,Эдриан Коутс
7,T9,ORG 418 422,HSBC
8,T10,ORG 521 528,Русалом
9,T11,ORG 535 550,Металлоинвестом


К сожалению, определились не все сущности

**SPACY NER**

In [38]:
nlp = spacy.load("ru_core_news_sm") 
ny_bb = data_text.text[8]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

**Самописный NER**

In [39]:
annot_files = sorted([file for file in os.listdir('/content/Collection3/') if file.endswith(".ann")])

In [40]:
annot_1 = pd.read_csv('/content/Collection3/001.ann', sep='\t')
annot_1.head(15)

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


In [41]:
#составляем списки токенов и аннотаций (из файла .ann делается словарь {слово : аннотация}

docs = []
for i in range(len(data_text)):
    words = []
    labels = []
    #подготавливаем текст
    text = data_text['text'][i]
    
    df = pd.read_csv('/content/Collection3/' + annot_files[i], delimiter='\t', header=None, quoting=csv.QUOTE_NONE)
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1]]
    df_ann['Segment'] = [loc[0] for loc in split_1]
  
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        segment = df_ann['Segment'][j]
        for tok in token:
            dic[tok] = segment

    for token in word_tokenize(text):
        if (token.lower() in dic.keys()):
            words.append(token)
            labels.append(dic[token.lower()])
        else:
            words.append(token)
            labels.append('OUT')
    
    docs.append([words, labels])

In [42]:
print(docs[0][0])
print(docs[0][1])

['Россия', 'рассчитывает', 'на', 'конструктивное', 'воздействие', 'США', 'на', 'Грузию', '04/08/2008', '12:08', 'МОСКВА', ',', '4', 'авг', '-', 'РИА', 'Новости', '.', 'Россия', 'рассчитывает', ',', 'что', 'США', 'воздействуют', 'на', 'Тбилиси', 'в', 'связи', 'с', 'обострением', 'ситуации', 'в', 'зоне', 'грузино-осетинского', 'конфликта', '.', 'Об', 'этом', 'статс-секретарь', '-', 'заместитель', 'министра', 'иностранных', 'дел', 'России', 'Григорий', 'Карасин', 'заявил', 'в', 'телефонном', 'разговоре', 'с', 'заместителем', 'госсекретаря', 'США', 'Дэниэлом', 'Фридом', '.', '``', 'С', 'российской', 'стороны', 'выражена', 'глубокая', 'озабоченность', 'в', 'связи', 'с', 'новым', 'витком', 'напряженности', 'вокруг', 'Южной', 'Осетии', ',', 'противозаконными', 'действиями', 'грузинской', 'стороны', 'по', 'наращиванию', 'своих', 'вооруженных', 'сил', 'в', 'регионе', ',', 'бесконтрольным', 'строительством', 'фортификационных', 'сооружений', "''", ',', '-', 'говорится', 'в', 'сообщении', '.', '`

In [43]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Россия	LOC
рассчитывает	OUT
на	OUT
конструктивное	OUT
воздействие	OUT
США	LOC
на	OUT
Грузию	LOC
04/08/2008	OUT
12:08	OUT
МОСКВА	LOC
,	OUT
4	OUT
авг	OUT
-	OUT
РИА	ORG
Новости	ORG
.	OUT
Россия	LOC
рассчитывает	OUT
,	OUT
что	OUT
США	LOC
воздействуют	OUT
на	OUT
Тбилиси	LOC
в	OUT
связи	OUT
с	OUT
обострением	OUT
ситуации	OUT
в	OUT
зоне	OUT
грузино-осетинского	OUT
конфликта	OUT
.	OUT
Об	OUT
этом	OUT
статс-секретарь	OUT
-	OUT
заместитель	OUT
министра	OUT
иностранных	OUT
дел	OUT
России	LOC
Григорий	PER
Карасин	PER
заявил	OUT
в	OUT
телефонном	OUT
разговоре	OUT
с	OUT
заместителем	OUT
госсекретаря	OUT
США	LOC
Дэниэлом	PER
Фридом	PER
.	OUT
``	OUT
С	OUT
российской	OUT
стороны	OUT
выражена	OUT
глубокая	OUT
озабоченность	OUT
в	OUT
связи	OUT
с	OUT
новым	OUT
витком	OUT
напряженности	OUT
вокруг	OUT
Южной	LOC
Осетии	LOC
,	OUT
противозаконными	OUT
действиями	OUT
грузинской	OUT
стороны	OUT
по	OUT
наращиванию	OUT
своих	OUT
вооруженных	OUT
сил	OUT
в	OUT
регионе	OUT
,	OUT
бесконтрольным	OUT
строительством	OUT


In [44]:
# делаем формат для sklearn_crf

df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(7)

,sent_id,data,entities
0,0,Россия,LOC
1,0,рассчитывает,OUT
2,0,на,OUT
3,0,конструктивное,OUT
4,0,воздействие,OUT
5,0,США,LOC
6,0,на,OUT


In [45]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None


getter = SentenceGetter(df)
#получаем список номеров преложений датасета 
sentences = getter.sentences

In [46]:
#преобразуем слова в признаки:
#для каждого слова-токена составляем словарь признаков, 
#определяем является ли оно начальным, конечным в предложении

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    # Добавляем левый контекст, если он есть
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True

    # Добавляем правый контекст, если он есть
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

#преобразуем предложения:
#каждое слово предложения преобразуем в признаки
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

#список всех лейблов предложения
def sent2labels(sent):
    return [label for token, label in sent]

#список всех токенов предложения
def sent2tokens(sent):
    return [token for token, label in sent]

In [47]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

**Передаем в модель токен с соседями**

In [48]:
#передаем в датасет токен с соседями (первый индекс -- предложение, 2 индекс -- слово)
X[2][7]

{'+1:word.lower()': 'арендуемом',
 '-1:word.lower()': 'изъяты',
 'bias': 1.0,
 'word.isdigit()': False,
 'word.lower()': 'в',
 'word[-2:]': 'в',
 'word[-3:]': 'в'}

In [49]:
X[1][5]

{'+1:word.lower()': 'отношении',
 '-1:word.lower()': 'политику',
 'bias': 1.0,
 'word.isdigit()': False,
 'word.lower()': 'в',
 'word[-2:]': 'в',
 'word[-3:]': 'в'}

In [50]:
len(X)

1000

In [51]:
# Разделим примерно на train и test выборки в соотношении 70/30
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [53]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [54]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Градиентный спуск с использованием метода L-BFGS
    c1=0.1, #Коэффициент для регуляризации L1
    c2=0.1, #Коэффициент для регуляризации L2
    max_iterations=1000, #Максимальное количество итераций
    all_possible_transitions=True, #Генерация объектов (не встречающихся в обучающих данных)
    verbose=True #Включение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 700/700 [00:01<00:00, 507.16it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 100262
Seconds required: 0.228

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.13  loss=120961.80 active=99395 feature_norm=1.00
Iter 2   time=0.08  loss=109347.22 active=95575 feature_norm=1.58
Iter 3   time=0.08  loss=107145.01 active=99094 feature_norm=1.46
Iter 4   time=0.07  loss=106029.26 active=98384 feature_norm=1.39
Iter 5   time=0.09  loss=104243.81 active=98958 feature_norm=1.46
Iter 6   time=0.12  loss=87839.95 active=98882 feature_norm=4.75
Iter 7   time=0.11  loss=85361.89 active=99770 feature_norm=5.13
Iter 8   time=0.08  loss=81261.67 active=100155 feature_norm=5.66
Iter 9   time=0.14  loss=78442.77 active=100119 feature_norm=6.60
Iter 

In [55]:
# Выводим список всех имеющихся уникальных сущностей
all_entities = sorted(df.entities.unique().tolist())

In [56]:
len(all_entities), all_entities

(4, ['LOC', 'ORG', 'OUT', 'PER'])

In [57]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities) 

0.947931606988054

In [61]:
print(metrics.flat_classification_report(y_test, y_pred, target_names =all_entities))

              precision    recall  f1-score   support

         LOC       0.94      0.80      0.86      2630
         ORG       0.81      0.62      0.70      4159
         OUT       0.96      0.99      0.97     64130
         PER       0.96      0.82      0.89      5107

    accuracy                           0.95     76026
   macro avg       0.92      0.81      0.85     76026
weighted avg       0.95      0.95      0.95     76026



**Передаем в модель только токен**

In [62]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [63]:
#собираем данные и метки
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [64]:
#передаем в датасет только токен
X[1][7]

{'bias': 1.0,
 'word.isdigit()': False,
 'word.lower()': 'беженцев',
 'word[-2:]': 'ев',
 'word[-3:]': 'цев'}

In [65]:
len(X)

1000

In [66]:
#разбиваем данные на train и test
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [67]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=True
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 700/700 [00:00<00:00, 1148.89it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 36624
Seconds required: 0.151

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.11  loss=121174.51 active=36458 feature_norm=1.00
Iter 2   time=0.06  loss=109579.59 active=35569 feature_norm=1.59
Iter 3   time=0.06  loss=107335.65 active=36407 feature_norm=1.46
Iter 4   time=0.06  loss=106401.84 active=36240 feature_norm=1.40
Iter 5   time=0.06  loss=105005.05 active=36341 feature_norm=1.45
Iter 6   time=0.06  loss=91614.11 active=36260 feature_norm=4.09
Iter 7   time=0.06  loss=89480.23 active=36532 feature_norm=4.99
Iter 8   time=0.06  loss=86408.94 active=36579 feature_norm=5.45
Iter 9   time=0.12  loss=84231.54 active=36565 feature_norm=6.36
Iter 10 

In [68]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities) 

0.9412212863059687

In [69]:
print(metrics.flat_classification_report(y_test, y_pred, labels=all_entities))

              precision    recall  f1-score   support

         LOC       0.92      0.76      0.83      2630
         ORG       0.80      0.56      0.66      4159
         OUT       0.95      0.99      0.97     64130
         PER       0.96      0.79      0.87      5107

    accuracy                           0.94     76026
   macro avg       0.91      0.78      0.83     76026
weighted avg       0.94      0.94      0.94     76026



**Вывод:** Spacy показывают результаты в задачах NER лучше, чем nltk.  
При использовании собственного алгоритма NER будет лучше передавать в алгоритм не только токен, но и его ближайших соседей (например, предыдущий и последующий токены) - так результат будет лучше.